In [19]:
class Intcode:
    
    def __init__(self, code, phase=None):
        
        self.intcode = [int(x) for x in code.strip().split(',')]
        self.i = 0
        
        self.input = []
        self.output = []
        
        self.loop = False
        
        if phase:
            self.input.append(phase)

    def __call__(self):
        self.compute()

    def set_input(self, input):
        self.input = input
        
    def get_output(self):
        return self.output

    
    def set_loop(self):
        self.loop = True
    
    
    def get(self, i=0, mode=0):
        
        if mode == 0:
            return self.intcode[i]
        
        elif mode == 1:
            return i
        
        else:
            raise ValueError

    def print_intcode(self):
        print(self.intcode)
            
    def compute(self):
        
        
        while self.i < len(self.intcode):
            
            opcode = int(str(self.intcode[self.i])[-2:])
            modes = str(self.intcode[self.i])[:-2].zfill(3)
            
            print(opcode)
            #print(opcode, modes, self.i, len(self.intcode))
            #self.print_intcode()
            #print(self.input, self.output)
            #print()
            
            
            
            
            nparam = getattr(self, "compute{}".format(opcode))(modes=modes)

            if opcode == 99:
                return
            
            
            self.i += nparam +1

            
            print(opcode, self.loop)
            
            if opcode == 4 and self.loop:
                return
            
            
            
    def compute1(self, nparam=3, modes="000"):
        self.intcode[self.get(self.intcode[self.i+3], mode=1)] = \
                    self.get(self.intcode[self.i+1], mode=int(modes[-1])) + \
                    self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            
        return nparam
        
    def compute2(self, nparam=3, modes="000"):
        self.intcode[self.get(self.intcode[self.i+3], mode=1)] = \
            self.get(self.intcode[self.i+1], mode=int(modes[-1])) * \
            self.get(self.intcode[self.i+2], mode=int(modes[-2]))
        
        return nparam
    
    def compute3(self, nparam=1, modes="1", keyboard=False):
        """input"""
        if keyboard:
            n = int(input("Enter integer:"))
        else:
            n = self.input.pop(0)
            
        self.intcode[self.get(self.i+1, mode=0)] = n
        return nparam
    
    def compute4(self, nparam=1, modes="0"):
        """output"""
        output = self.get(self.intcode[self.i+1], mode=int(modes[-1]))
        print(">", output)
        self.output.append(output)
        return nparam
    
    def compute5(self, nparam=2, modes="00"):
        """jump-if-true"""
        
        if self.get(self.intcode[self.i+1], mode=int(modes[-1])):
            self.i = self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            return -1       
        
        return nparam
    
    def compute6(self, nparam=2, modes="00"):
        """jump-if-false"""
        
        if not self.get(self.intcode[self.i+1], mode=int(modes[-1])):
            self.i = self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            return -1       
        
        return nparam
    
    def compute7(self, nparam=3, modes="000"):
        """less than"""

        if self.get(self.intcode[self.i+1], mode=int(modes[-1])) < self.get(self.intcode[self.i+2], mode=int(modes[-2])):
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 1
        else:
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 0
        return nparam
    
    def compute8(self, nparam=3, modes="000"):
        """equals"""
        if self.get(self.intcode[self.i+1], mode=int(modes[-1])) == self.get(self.intcode[self.i+2], mode=int(modes[-2])):
            
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 1
        else:
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 0
        return nparam
    
    def compute99(self, nparam=1, modes="0"):

        return 
      

In [9]:
def max_signal(setting, code):
    output = 0
    for phase in setting:
        #print(phase, output)
        #print("................")
        ic = Intcode(code)
        ic.set_input([phase, output])
        ic.compute()
        output, = ic.get_output()
    return output


setting = [4,3,2,1,0]
code = "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0"
print(max_signal(setting, code))

setting = [0,1,2,3,4]
code = "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0"
print(max_signal(setting, code))


> 4
> 43
> 432
> 4321
> 43210
43210
> 5
> 54
> 543
> 5432
> 54321
54321


In [10]:
import itertools

with open("input.txt") as f:
    incode = f.read()

max_ = 0
max_setting = None
for setting in itertools.permutations(range(5)):
    value = max_signal(setting, incode)
    if value > max_:
        max_ = value
        max_setting = setting

print(max_, max_setting)
    

> 56
> 1212
> 9732
> 29213
> 146065
> 56
> 1212
> 9732
> 48660
> 145997
> 56
> 1212
> 3653
> 29260
> 146300
> 56
> 1212
> 3653
> 18265
> 146156
> 56
> 1212
> 6060
> 48516
> 145565
> 56
> 1212
> 6060
> 18197
> 145612
> 56
> 484
> 9772
> 29333
> 146665
> 56
> 484
> 9772
> 48860
> 146597
> 56
> 484
> 1469
> 29472
> 147360
> 56
> 484
> 1469
> 7345
> 146992
> 56
> 484
> 2420
> 48492
> 145493
> 56
> 484
> 2420
> 7277
> 145632
> 56
> 185
> 3792
> 30372
> 151860
> 56
> 185
> 3792
> 18960
> 151716
> 56
> 185
> 1516
> 30412
> 152060
> 56
> 185
> 1516
> 7580
> 151692
> 56
> 185
> 925
> 18592
> 148772
> 56
> 185
> 925
> 7436
> 148812
> 56
> 280
> 5692
> 45572
> 136733
> 56
> 280
> 5692
> 17093
> 136780
> 56
> 280
> 2276
> 45612
> 136853
> 56
> 280
> 2276
> 6845
> 136992
> 56
> 280
> 857
> 17232
> 137892
> 56
> 280
> 857
> 6892
> 137932
> 92
> 1160
> 9316
> 27965
> 139825
> 92
> 1160
> 9316
> 46580
> 139757
> 92
> 1160
> 3497
> 28012
> 140060
> 92
> 1160
> 3497
> 17485
> 139916
> 92
> 1160
> 5800
>

```--- Part Two ---```

In [25]:
class Intcode:
    
    def __init__(self, code, loop=False, phase=None):
        
        self.intcode = [int(x) for x in code.strip().split(',')]
        self.i = 0
        
        self.input = []
        self.output = []
        
        self.loop = loop
        self.halt = None
        
        if phase is not None:
            self.input.append(phase)
        
        

    def __call__(self):
        self.compute()

    def set_input(self, input):
        self.input = input
        
        
    def get_output(self):
        return self.output

    
    def set_loop(self):
        self.loop = True
    
    
    def get(self, i=0, mode=0):
        
        if mode == 0:
            return self.intcode[i]
        
        elif mode == 1:
            return i
        
        else:
            raise ValueError

    def print_intcode(self):
        print(self.intcode)
            
    def compute(self):
        
        
        while self.i < len(self.intcode):
            
            opcode = int(str(self.intcode[self.i])[-2:])
            modes = str(self.intcode[self.i])[:-2].zfill(3)
            
            #print(opcode, modes)
            #print(opcode, modes, self.i, len(self.intcode))
            #self.print_intcode()
            #print(self.input, self.output)
            #print()
            
            
            
            nparam = getattr(self, "compute{}".format(opcode))(modes=modes)

            if opcode == 99:
                return
            
            
            self.i += nparam +1

            if opcode == 4 and self.loop:
                return
            
            
            
    def compute1(self, nparam=3, modes="000"):
        self.intcode[self.get(self.intcode[self.i+3], mode=1)] = \
                    self.get(self.intcode[self.i+1], mode=int(modes[-1])) + \
                    self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            
        return nparam
        
    def compute2(self, nparam=3, modes="000"):
        self.intcode[self.get(self.intcode[self.i+3], mode=1)] = \
            self.get(self.intcode[self.i+1], mode=int(modes[-1])) * \
            self.get(self.intcode[self.i+2], mode=int(modes[-2]))
        
        return nparam
    
    def compute3(self, nparam=1, modes="1", keyboard=False):
        """input"""
        if keyboard:
            n = int(input("Enter integer:"))
        else:
            n = self.input.pop(0)
            
        self.intcode[self.get(self.i+1, mode=0)] = n
        return nparam
    
    def compute4(self, nparam=1, modes="0"):
        """output"""
        output = self.get(self.intcode[self.i+1], mode=int(modes[-1]))
        #print(">", output)
        self.output.append(output)
        return nparam
    
    def compute5(self, nparam=2, modes="00"):
        """jump-if-true"""
        
        if self.get(self.intcode[self.i+1], mode=int(modes[-1])):
            self.i = self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            return -1       
        
        return nparam
    
    def compute6(self, nparam=2, modes="00"):
        """jump-if-false"""
        
        if not self.get(self.intcode[self.i+1], mode=int(modes[-1])):
            self.i = self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            return -1       
        
        return nparam
    
    def compute7(self, nparam=3, modes="000"):
        """less than"""

        if self.get(self.intcode[self.i+1], mode=int(modes[-1])) < self.get(self.intcode[self.i+2], mode=int(modes[-2])):
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 1
        else:
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 0
        return nparam
    
    def compute8(self, nparam=3, modes="000"):
        """equals"""
        if self.get(self.intcode[self.i+1], mode=int(modes[-1])) == self.get(self.intcode[self.i+2], mode=int(modes[-2])):
            
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 1
        else:
            self.intcode[self.get(self.intcode[self.i+3], mode=1)] = 0
        return nparam
    
    def compute99(self, nparam=1, modes="0"):
        self.halt = True
        return 
      

In [23]:
def max_signal2(setting, code):


    amps = [Intcode(code, phase=phase, loop=True) for phase in setting]

    output = 0
    i = 0
    
    while True:
        
        idx = i%len(amps)
        #print("idx", idx)
        
        amp = amps[idx]
        #print(amp.input)
        amp.input.append(output)
        
        #print(amp.input)
        
        amp.compute()
        
        if amp.halt:
            break
        
        output = amp.output.pop()          
        
        i += 1
        
    return output


In [15]:
setting = [9,8,7,6,5]
code = "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"
max_signal2(setting, code)

139629729

In [16]:
setting = [9,7,8,5,6]
code = "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10"
max_signal2(setting, code)

18216

In [24]:
setting = [3, 2, 0, 1, 4] 
with open("input.txt") as f:
    incode = f.read().strip()
max_signal2(setting, incode)

> 17
> 172
> 2120
> 42492
> 212460


212460

In [26]:
import itertools

with open("input.txt") as f:
    incode = f.read()

max_ = 0
max_setting = None
for setting in itertools.permutations(range(5, 10)):
    
    value = max_signal2(setting, incode)
    print(setting, value)
    if value > max_:
        max_ = value
        max_setting = setting

print(max_, max_setting)
    

(5, 6, 7, 8, 9) 18429706
(5, 6, 7, 9, 8) 17872556
(5, 6, 8, 7, 9) 19552258
(5, 6, 8, 9, 7) 19556385
(5, 6, 9, 7, 8) 17876715
(5, 6, 9, 8, 7) 18441955
(5, 7, 6, 8, 9) 18429676
(5, 7, 6, 9, 8) 17872526
(5, 7, 8, 6, 9) 19552348
(5, 7, 8, 9, 6) 19556444
(5, 7, 9, 6, 8) 17876622
(5, 7, 9, 8, 6) 18441980
(5, 8, 6, 7, 9) 21797730
(5, 8, 6, 9, 7) 21801857
(5, 8, 7, 6, 9) 21797716
(5, 8, 7, 9, 6) 21801812
(5, 8, 9, 6, 7) 21810049
(5, 8, 9, 7, 6) 21810034
(5, 9, 6, 7, 8) 17884907
(5, 9, 6, 8, 7) 18450147
(5, 9, 7, 6, 8) 17884876
(5, 9, 7, 8, 6) 18450234
(5, 9, 8, 6, 7) 19580993
(5, 9, 8, 7, 6) 19580978
(6, 5, 7, 8, 9) 16070478
(6, 5, 7, 9, 8) 15513328
(6, 5, 8, 7, 9) 17193030
(6, 5, 8, 9, 7) 17197157
(6, 5, 9, 7, 8) 15517487
(6, 5, 9, 8, 7) 16082727
(6, 7, 5, 8, 9) 14890846
(6, 7, 5, 9, 8) 14333696
(6, 7, 8, 5, 9) 14899422
(6, 7, 8, 9, 5) 14313728
(6, 7, 9, 5, 8) 13748038
(6, 7, 9, 8, 5) 13756358
(6, 8, 5, 7, 9) 17210310
(6, 8, 5, 9, 7) 17214437
(6, 8, 7, 5, 9) 16030670
(6, 8, 7, 9, 5) 15444976
